In [1]:
import numpy as np
import amstrax
import strax
import matplotlib.pyplot as plt
import pandas as pd
from pymongo import MongoClient
%matplotlib inline
%load_ext line_profiler

/Users/petergaemers/Desktop/python/straxen/straxen/plugins/fax.py:142: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "add_current" failed type inference due to: Untyped global name '_template_length': cannot determine Numba type of <class 'numba.ir.UndefinedType'>

File "../../straxen/straxen/plugins/fax.py", line 153:
    def add_current(_photon_timing_start, _reminder, _photon_gain, pulse):
        <source elided>
        #         """
        pulse[_photon_timing_start:_photon_timing_start + _template_length] += \
        ^

  @guvectorize([(int64, int64, float64, float64[:])], '(), (), (), (n)')
/anaconda3/envs/amstrax/lib/python3.6/site-packages/numba-0.44.1-py3.6-macosx-10.7-x86_64.egg/numba/compiler.py:725: NumbaWarning: Function "add_current" was compiled in object mode without forceobj=True.

File "../../straxen/straxen/plugins/fax.py", line 143:
    @guvectorize([(int64, int64, float64, float64[:])], '(), (), (), (n)')


In [2]:
%matplotlib inline

In [3]:
# run_id = '190516_095707'_
run_id = '190610_085915'

In [4]:
st = strax.Context(storage=[strax.DataDirectory('./data'),
                           strax.DataDirectory('./raw',readonly= True),
                           ],
                  register=[amstrax.RecordsFromMongo, amstrax.PulseProcessing, amstrax.Peaks, amstrax.TriggerPeaks,
                            amstrax.PeakBasics,amstrax.PeakClassification,amstrax.PeakPositions,amstrax.NCompeting,
                            amstrax.Events,amstrax.EventBasics, amstrax.EventPositions, amstrax.EventInfo],
                    register_all=amstrax,
#                    config=dict(pax_raw_dir='/data/xenon/xams/run10/raw/data/'),
                  )

AttributeError: module 'amstrax' has no attribute 'EventPositions'

In [ ]:
st.get_df(run_id, 'event_info')

In [ ]:
# !ls ./data

In [ ]:
! rm -r ./data/190610_085915*

In [ ]:
# r = st.get_array(run_id, 'records')

In [ ]:
from SiPMdata import *

In [ ]:
class EventPositions(strax.LoopPlugin):
    depends_on = ('events','event_basics','peaks', 'peak_classification')
    dtype = [
        ('xr', np.float32,
         'Interaction x-position'),
        ('yr', np.float32,
         'Interaction y-position'),
    ]

    def setup(self):
        # z position of the in-plane SiPMs
        z_plane = 10
        # radius of the cyinder for SiPMs at the side
        r_cylinder = 22
        # radius of a SiPM - I assume circular SiPMs with a radius to make the area correspond to a 3x3mm2 square.
        r_sipm = 1.6925
        # build geometry
        geo = GeoParameters(z_plane=z_plane, r_cylinder=r_cylinder, r_sipm=r_sipm)

        sipm = SiPM(type="plane", position=[0, -15, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        sipm = SiPM(type="plane", position=[-13, -7.5, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        sipm = SiPM(type="plane", position=[13, -7.5, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        sipm = SiPM(type="plane", position=[-4, 0, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        sipm = SiPM(type="plane", position=[4, 0, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        sipm = SiPM(type="plane", position=[-13, 7.5, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        sipm = SiPM(type="plane", position=[13, 7.5, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        
        self.geo = geo

    def compute_loop(self, events, peaks):
        result = dict()
        
        if not len(peaks):
            return result
        
        s2_index = events['s2_index']
        if s2_index == -1 or s2_index > len(peaks[(peaks['type']==2)]) -1 :
            return result
        
        s2_peak = peaks[(peaks['type']==2)][s2_index]
        for i, area in enumerate(s2_peak['area_per_channel'][:7]):
            self.geo.sipms[i].set_number_of_hits(area)

        posrec = Reconstruction(self.geo)
        pos = posrec.reconstruct_position('CHI2')
        for key in ['xr','yr']:
            result[key] = pos[key]
        return result


In [ ]:
ep = st.get_array(run_id,'event_positions',
                 register = EventPositions)

In [ ]:
ep